In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [5]:
movies_path = '/kaggle/input/the-movies-dataset/movies_metadata.csv'
df_movies = pd.read_csv(movies_path, low_memory=False)
df_movies = df_movies[['id', 'title', 'overview']]
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')
df_movies.dropna(subset=['id', 'title', 'overview'], inplace=True)
df_movies['id'] = df_movies['id'].astype(int)

ratings_path = '/kaggle/input/the-movies-dataset/ratings_small.csv'
links_path = '/kaggle/input/the-movies-dataset/links_small.csv'
df_ratings = pd.read_csv(ratings_path)
df_links = pd.read_csv(links_path)

print("Kích thước Movies Dataset:", df_movies.shape)
print("Kích thước Ratings (MovieLens):", df_ratings.shape)

# --- Hợp nhất dữ liệu ---
# MovieLens dùng 'movieId', The Movies Dataset dùng 'id' và 'tmdbId'.
# df_links giúp kết nối chúng.
df_links.dropna(subset=['tmdbId'], inplace=True)
df_links['tmdbId'] = df_links['tmdbId'].astype(int)

# Hợp nhất ratings với links để có tmdbId
df_ratings = pd.merge(df_ratings, df_links, on='movieId')

# Hợp nhất kết quả với df_movies để có nội dung
# Đổi tên cột 'id' trong df_movies để hợp nhất
df_movies.rename(columns={'id': 'tmdbId'}, inplace=True)
df_full = pd.merge(df_ratings, df_movies, on='tmdbId')

# Loại bỏ các cột không cần thiết sau khi hợp nhất
df_full = df_full[['userId', 'movieId', 'rating', 'title', 'overview']]

# Loại bỏ các phim trùng lặp để ma trận TF-IDF không bị lỗi index
df_full.drop_duplicates(subset='movieId', inplace=True)
df_full.reset_index(drop=True, inplace=True)

print("\nKích thước dữ liệu sau khi hợp nhất và làm sạch:", df_full.shape)
display(df_full.head())

Kích thước Movies Dataset: (44506, 3)
Kích thước Ratings (MovieLens): (100004, 4)

Kích thước dữ liệu sau khi hợp nhất và làm sạch: (9013, 5)


,userId,movieId,rating,title,overview
0,1,31,2.5,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
1,1,1029,3.0,Dumbo,Dumbo is a baby elephant born with oversized e...
2,1,1061,3.0,Sleepers,Two gangsters seek revenge on the state jail w...
3,1,1129,2.0,Escape from New York,"In 1997, the island of Manhattan has been wall..."
4,1,1172,4.0,Cinema Paradiso,"A filmmaker recalls his childhood, when he fel..."


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_full['overview'])

print("Kích thước ma trận TF-IDF:", tfidf_matrix.shape)

Kích thước ma trận TF-IDF: (9013, 29636)


In [9]:
def build_user_profile(user_id, df_ratings, tfidf_matrix):
    """
    Xây dựng vector profile cho một user dựa trên các phim họ đã thích.
    
    Args:
        user_id (int): ID của người dùng.
        df_ratings (pd.DataFrame): DataFrame chứa rating của tất cả user.
        tfidf_matrix (scipy.sparse.csr_matrix): Ma trận TF-IDF của tất cả phim.
    
    Returns:
        scipy.sparse.csr_matrix: Một vector thưa đại diện cho profile của user.
    """
    # Lọc ra các rating của user_id
    user_data = df_ratings[df_ratings['userId'] == user_id]
    
    # Lọc ra các phim mà user này đã rate cao (>= 4.0)
    liked_movies = user_data[user_data['rating'] >= 4.0]
    
    if liked_movies.empty:
        print(f"User {user_id} không có phim nào được đánh giá cao.")
        # Trả về một vector 0 nếu không có dữ liệu
        return np.zeros(tfidf_matrix.shape[1])

    # Lấy index của các phim đã thích trong df_full
    # .index.values sẽ trả về index của DataFrame, khớp với index của tfidf_matrix
    liked_indices = liked_movies.index.values
    
    # Lấy các vector TF-IDF của các phim đã thích
    liked_vectors = tfidf_matrix[liked_indices]
    
    # --- Xây dựng profile ---
    # Cách 1: Trung bình cộng (đơn giản)
    # user_profile = liked_vectors.mean(axis=0)

    # Cách 2: Trung bình có trọng số (tốt hơn)
    # Phim rate 5 sao nên có ảnh hưởng lớn hơn phim rate 4 sao
    ratings_weights = liked_movies['rating'].values
    
    # Nhân mỗi vector phim với rating tương ứng
    # .T để nhân broadcasting đúng chiều
    weighted_vectors = liked_vectors.multiply(ratings_weights[:, np.newaxis])
    
    # Tính trung bình của các vector có trọng số
    user_profile = weighted_vectors.mean(axis=0)

    user_profile = np.asarray(user_profile)

    # Chuẩn hóa lại vector profile để nó có độ dài là 1 (quan trọng cho cosine similarity)
    from sklearn.preprocessing import normalize
    user_profile_normalized = normalize(user_profile)
    
    return user_profile_normalized

# --- Thử nghiệm xây dựng profile cho user 1 ---
user_id_to_test = 1
user_profile = build_user_profile(user_id_to_test, df_full, tfidf_matrix)

print(f"Đã tạo profile cho User {user_id_to_test}.")
print("Kích thước vector profile:", user_profile.shape)

Đã tạo profile cho User 1.
Kích thước vector profile: (1, 29636)
